In [1]:
from model import SiameseBiLSTM
from inputHandler import word_embed_meta_data, create_test_data
from config import siamese_config

from operator import itemgetter
from keras.models import load_model
import pandas as pd
from keras.models import load_model
from keras.utils import plot_model

########################################
############ Data Preperation ##########
########################################


df = pd.read_csv('sample_data.csv')

sentences1 = list(df['sentences1'])
sentences2 = list(df['sentences2'])
is_similar = list(df['is_similar'])
#del df


####################################
######## Word Embedding ############
####################################


# creating word embedding meta data for word embedding 
tokenizer, embedding_matrix = word_embed_meta_data(sentences1 + sentences2,  siamese_config['EMBEDDING_DIM'])

embedding_meta_data = {
	'tokenizer': tokenizer,
	'embedding_matrix': embedding_matrix
}

## creating sentence pairs
sentences_pair = [(x1, x2) for x1, x2 in zip(sentences1, sentences2)]
#del sentences1
#del sentences2


##########################
######## Training ########
##########################

from config import siamese_config


class Configuration(object):
    """Dump stuff here"""

CONFIG = Configuration()
CONFIG.embedding_dim = siamese_config['EMBEDDING_DIM']
CONFIG.max_sequence_length = siamese_config['MAX_SEQUENCE_LENGTH']
CONFIG.number_lstm_units = siamese_config['NUMBER_LSTM']
CONFIG.rate_drop_lstm = siamese_config['RATE_DROP_LSTM']
CONFIG.number_dense_units = siamese_config['NUMBER_DENSE_UNITS']
CONFIG.activation_function = siamese_config['ACTIVATION_FUNCTION']
CONFIG.rate_drop_dense = siamese_config['RATE_DROP_DENSE']
CONFIG.validation_split_ratio = siamese_config['VALIDATION_SPLIT']

siamese = SiameseBiLSTM(CONFIG.embedding_dim , CONFIG.max_sequence_length, CONFIG.number_lstm_units , CONFIG.number_dense_units, 
					    CONFIG.rate_drop_lstm, CONFIG.rate_drop_dense, CONFIG.activation_function, CONFIG.validation_split_ratio)

best_model_path = siamese.train_model(sentences_pair, is_similar, embedding_meta_data, model_save_directory='./')


########################
###### Testing #########
########################

model = load_model(best_model_path)
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

test_sentence_pairs = [('What can make Physics easy to learn?','How can you make physics easy to learn?'),
					   ('How many times a day do a clocks hands overlap?','What does it mean that every time I look at the clock the numbers are the same?')]

test_data_x1, test_data_x2, leaks_test = create_test_data(tokenizer,test_sentence_pairs,  siamese_config['MAX_SEQUENCE_LENGTH'])

preds = list(model.predict([test_data_x1, test_data_x2, leaks_test], verbose=1).ravel())
results = [(x, y, z) for (x, y), z in zip(test_sentence_pairs, preds)]
results.sort(key=itemgetter(2), reverse=True)
print(results)














Embedding matrix shape: (3052, 50)
Null word embeddings: 1
Epoch 1/200
8/8 [==============================] - 14s 546ms/step - loss: 0.8477 - acc: 0.4962 - val_loss: 0.6691 - val_acc: 0.4898
Epoch 2/200
8/8 [==============================] - 1s 190ms/step - loss: 0.8162 - acc: 0.5364 - val_loss: 0.6578 - val_acc: 0.5918
Epoch 3/200
8/8 [==============================] - 2s 191ms/step - loss: 0.7560 - acc: 0.5518 - val_loss: 0.6446 - val_acc: 0.5918
Epoch 4/200
8/8 [==============================] - 1s 189ms/step - loss: 0.6746 - acc: 0.6167 - val_loss: 0.6429 - val_acc: 0.5918
Epoch 5/200
8/8 [==============================] - 1s 188ms/step - loss: 0.7457 - acc: 0.5617 - val_loss: 0.6426 - val_acc: 0.5918
Epoch 6/200
8/8 [==============================] - 2s 199ms/step - loss: 0.6816 - acc: 0.6309 - val_loss: 0.6403 - val_acc: 0.5918
Epoch 7/200
8/8 [==============================] - 2s 189ms/step - loss: 0.6351 - acc: 0.6270 - val_loss: 0.6329 - val_acc: 0.5918
Epoch 8/200
8/8 [======